In [1]:
// Load the EnvConfigHelper class from our project
#load "../EnvConfigHelper.cs"

  // Load configuration from .env file
  var modelDeploymentName = EnvConfigHelper.AzureOpenAIDeploymentName;
  var azureOpenAIEndpoint = EnvConfigHelper.AzureOpenAIEndpoint;
  var azureOpenAIKey = EnvConfigHelper.AzureOpenAIKey;


// Display loaded configuration (without sensitive data)
Console.WriteLine("🔑 Configuration loaded successfully!");
Console.WriteLine($"Azure OpenAI Endpoint: {azureOpenAIEndpoint}");
Console.WriteLine($"Deployment Name: {modelDeploymentName}");
Console.WriteLine($"API Key: {(string.IsNullOrEmpty(azureOpenAIKey) ? "❌ Not loaded" : "✅ Loaded (hidden)")}");

✅ Loaded .env from: d:\VSC\semantic-kernel-in-action-1-fundamentals-3836112\src\.env
🔑 Configuration loaded successfully!
Azure OpenAI Endpoint: https://dis-openai-0705.openai.azure.com/
Deployment Name: gpt-4.1
API Key: ✅ Loaded (hidden)


In [16]:
#r "nuget: Microsoft.SemanticKernel, *"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.61.0-preview"
#r "nuget: Microsoft.SemanticKernel.PromptTemplates.Handlebars, *"



using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Plugins.Core;
using Microsoft.SemanticKernel.PromptTemplates.Handlebars;
using System.ComponentModel;
using System.Net.Http;
using System.IO;


Installed Packages Microsoft.SemanticKernel, 1.61.0 Microsoft.SemanticKernel.Plugins.Core, 1.61.0-preview Microsoft.SemanticKernel.PromptTemplates.Handlebars, 1.61.0

In [17]:
public class WhatTimeIsIt
{
  [KernelFunction, Description("Get the current time")]
  public string Time(IFormatProvider? formatProvider = null) =>
      DateTimeOffset.Now.ToString("hh:mm:ss tt", formatProvider);
}

In [9]:
async Task Execute()
{
    try
    {
        var builder = Kernel.CreateBuilder();
        builder.Services.AddAzureOpenAIChatCompletion(
            modelDeploymentName,
            azureOpenAIEndpoint,
            azureOpenAIKey,
            modelId: "gpt-4.1"
        );

        builder.Plugins.AddFromType<WhatTimeIsIt>();
        var kernel = builder.Build();
  

       // Create agenda
        List<string> todaysCalendar = ["8am - wakeup", "9am - work", "12am - lunch", "1pm - work", "6pm - exercise", "7pm - study", "10pm - sleep"];

        var handlebarsTemplate = @"
                        Please explain in a fun way the day agenda
                        {{ set ""dayAgenda"" (todaysCalendar)}}
                        {{ set ""whatTimeIsIt"" (WhatTimeIsIt-Time) }}

                        {{#each dayAgenda}}
                            Explain what you are doing at {{this}} in a fun way.
                        {{/each}}

                        Explain what you will be doing next at {{whatTimeIsIt}} in a fun way.";

        // Create handlebars template for intent
        var handlebarsFunction = kernel.CreateFunctionFromPrompt(
            new PromptTemplateConfig()
            {
            Template = handlebarsTemplate,
            TemplateFormat = "handlebars"
            },
            new HandlebarsPromptTemplateFactory()
        );

        var todaysFunCalendar = await kernel.InvokeAsync(
            handlebarsFunction,
            new() {
            { "todaysCalendar", todaysCalendar }
            }
        );

        Console.WriteLine($"Today's fun calendar:  {todaysFunCalendar}");
        }
        catch (Exception ex)
        {
            // Output any errors to the cell
            Console.WriteLine($"Error: {ex.Message}");
        }
}


In [ ]:
async Task Execute()
{
    try
    {
        // Validate required configuration
        if (string.IsNullOrEmpty(azureOpenAIKey) || string.IsNullOrEmpty(azureOpenAIEndpoint))
        {
            Console.WriteLine("❌ Azure OpenAI configuration is missing. Please check your .env file.");
            return;
        }

        var builder = Kernel.CreateBuilder();
        
        // Use a valid model ID (gpt-4 instead of gpt-4.1)
        builder.Services.AddAzureOpenAIChatCompletion(
            modelDeploymentName,
            azureOpenAIEndpoint,
            azureOpenAIKey,
            modelId: "gpt-4" // Fixed model ID
        );

        builder.Plugins.AddFromType<WhatTimeIsIt>();
        var kernel = builder.Build();

        // Create agenda with corrected time format
        List<string> todaysCalendar = [
            "8am - wake up", 
            "9am - work", 
            "12pm - lunch",  // Fixed: was 12am
            "1pm - work", 
            "6pm - exercise", 
            "7pm - study", 
            "10pm - sleep"
        ];

        var handlebarsTemplate = @"
            Please explain in a fun way the day agenda:
            {{ set ""dayAgenda"" (todaysCalendar) }}
            {{ set ""currentTime"" (WhatTimeIsIt-Time) }}

            {{#each dayAgenda}}
            🎯 {{this}} - Make this sound exciting and engaging!
            {{/each}}

            🕐 Current time: {{currentTime}}
            What should you be doing right now? Make it sound awesome!";

        // Create handlebars function
        var handlebarsFunction = kernel.CreateFunctionFromPrompt(
            new PromptTemplateConfig()
            {
                Template = handlebarsTemplate,
                TemplateFormat = "handlebars"
            },
            new HandlebarsPromptTemplateFactory()
        );

        Console.WriteLine("🚀 Generating your fun daily agenda...");
        
        var todaysFunCalendar = await kernel.InvokeAsync(
            handlebarsFunction,
            new() {
                { "todaysCalendar", todaysCalendar }
            }
        );

        Console.WriteLine($"📅 Today's Fun Calendar:\n{todaysFunCalendar}");
    }
    catch (HttpRequestException httpEx)
    {
        Console.WriteLine($"🌐 Network Error: {httpEx.Message}");
        Console.WriteLine("Please check your Azure OpenAI endpoint and network connection.");
    }
    catch (UnauthorizedAccessException authEx)
    {
        Console.WriteLine($"🔐 Authentication Error: {authEx.Message}");
        Console.WriteLine("Please verify your Azure OpenAI API key.");
    }
    catch (Exception ex)
    {
        Console.WriteLine($"❌ Error: {ex.Message}");
        if (ex.InnerException != null)
        {
            Console.WriteLine($"   Inner Exception: {ex.InnerException.Message}");
        }
    }
}

The active selection demonstrates the core functionality of a Semantic Kernel application that creates an AI-powered daily agenda generator using Handlebars templating. This code showcases how to combine plugins, templates, and AI to transform a simple schedule into an engaging, personalized experience.

The code begins by registering the `WhatTimeIsIt` plugin with the kernel using `builder.Plugins.AddFromType<WhatTimeIsIt>()`. This plugin provides time-related functionality that will be accessible within the Handlebars template. After building the kernel, a sample daily calendar is created as a list of strings representing typical daily activities with their scheduled times. The calendar includes corrected time formatting, notably fixing "12am" to "12pm" for lunch, which demonstrates attention to realistic scheduling.

The Handlebars template is the heart of this implementation, defining how the AI should process and present the calendar data. The template uses Handlebars syntax with `{{ set }}` directives to assign variables - `dayAgenda` receives the calendar data, while `currentTime` captures the current time from the plugin. The `{{#each}}` block iterates through each calendar item, instructing the AI to make each activity sound "exciting and engaging" with emoji formatting. The template concludes by asking the AI to suggest what the user should be doing at the current time in an enthusiastic manner.

The code then creates a kernel function from this prompt template using `CreateFunctionFromPrompt()`. This method takes a `PromptTemplateConfig` that specifies the template content and format ("handlebars"), along with a `HandlebarsPromptTemplateFactory` to handle the template processing. When the function is invoked with `kernel.InvokeAsync()`, it passes the calendar data as arguments, allowing the AI to generate a fun, personalized interpretation of the daily schedule.

This approach demonstrates several key concepts: plugin integration for dynamic data (current time), template-driven AI prompts for consistent formatting, and the separation of data (calendar) from presentation logic (template). The result is a flexible system that can transform any daily schedule into an engaging, AI-generated narrative while maintaining the structure and timing of the original agenda.

In [19]:
await Execute();

🚀 Generating your fun daily agenda...
📅 Today's Fun Calendar:
Absolutely! Let’s make your day sound like the most epic adventure ever:

---
🎯 **8am – Wake Up!**  
Rise and shine, champion! The world is waiting for your first amazing move—stretch, yawn, and get ready to launch into greatness like a rocket blasting off at sunrise!

🎯 **9am – Work!**  
Time to suit up and enter productivity mode! You’re a hero on a mission—every email answered, every task tackled brings you closer to workday victory. Unleash your inner productivity ninja!

🎯 **12pm – Lunch!**  
Sound the trumpets! It’s taste-bud adventure time. Take your taste buds to flavor town, refuel like a legend, and savor every bite—because lunch is your delicious pit stop on the racetrack of life.

🎯 **1pm – Work!**  
Round two, game face on! Jump back into action, conquer challenges, and check off that to-do list with the power and focus of a superhero in their prime.

🎯 **6pm – Exercise!**  
Suit up in your athletic gear—this is